In [ ]:
import os
from multiprocessing import cpu_count
import random
import tensorflow as tf
from tensorflow.keras import layers, models, mixed_precision
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
import json
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize
import seaborn as sns

In [ ]:

def load_and_preprocess_data():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

    val_images = train_images[:5000]
    val_labels = train_labels[:5000]
    train_images = train_images[5000:]
    train_labels = train_labels[5000:]

    train_images = train_images.astype('float32') / 255
    val_images = val_images.astype('float32') / 255
    test_images = test_images.astype('float32') / 255

    train_labels = to_categorical(train_labels, 10)
    val_labels = to_categorical(val_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    return train_images, train_labels, val_images, val_labels, test_images, test_labels

def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(10, activation='softmax', dtype='float32')
    ])
    return model

def plot_learning_curves(history):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history['accuracy'], label='Train Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history['loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.savefig('output_phase1/1/learning_curves.png')
    plt.close()

def plot_confusion_matrix(model, test_images, test_labels):
    # پیش‌بینی کلاس‌ها
    y_pred = model.predict(test_images)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(test_labels, axis=1)

    # محاسبه ماتریس درهم‌ریختگی
    cm = confusion_matrix(y_true, y_pred_classes)

    # رسم ماتریس
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=range(10), yticklabels=range(10))
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig('output_phase1/1/confusion_matrix.png')
    plt.close()

def plot_roc_curve(model, test_images, test_labels):
    # پیش‌بینی احتمالات
    y_pred = model.predict(test_images)

    # تبدیل برچسب‌ها به فرمت باینری برای هر کلاس
    y_true_bin = label_binarize(np.argmax(test_labels, axis=1), classes=range(10))

    # محاسبه ROC برای هر کلاس
    plt.figure(figsize=(10, 8))
    for i in range(10):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Each Class')
    plt.legend(loc="lower right")
    plt.savefig('output_phase1/1/roc_curve.png')
    plt.close()

def visualize_predictions(model, test_images, test_labels, num_samples=20):
    # انتخاب تصادفی نمونه‌ها
    indices = random.sample(range(len(test_images)), num_samples)
    sample_images = test_images[indices]
    sample_labels = np.argmax(test_labels[indices], axis=1)

    # پیش‌بینی مدل
    predictions = model.predict(sample_images)
    predicted_classes = np.argmax(predictions, axis=1)

    # تنظیمات نمودار
    plt.figure(figsize=(15, 10))
    for i in range(num_samples):
        plt.subplot(4, 5, i+1)
        plt.imshow(sample_images[i])

        # تعیین رنگ متن بر اساس صحت پیش‌بینی
        color = 'green' if predicted_classes[i] == sample_labels[i] else 'red'

        plt.title(f"True: {class_names[sample_labels[i]]}\nPred: {class_names[predicted_classes[i]]}",
                 color=color)
        plt.axis('off')

    plt.tight_layout()
    plt.savefig('output_phase1/2/predictions_visualization.png')
    plt.close()

def train_model(model, train_images, train_labels, val_images, val_labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    train_dataset = train_dataset.shuffle(10000).batch(512).prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
    val_dataset = val_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    # فقط ذخیره بهترین مدل بدون توقف زودرس
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        'output_phase1/best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    history = model.fit(
        train_dataset,
        epochs=30,  # حتماً 30 epoch اجرا شود
        validation_data=val_dataset,
        callbacks=[checkpoint],
        verbose=1
    )
    return history.history

def save_results(history, test_results, y_true, y_pred_classes):
    # محاسبه Precision, Recall, F1-Score
    precision = precision_score(y_true, y_pred_classes, average='weighted')
    recall = recall_score(y_true, y_pred_classes, average='weighted')
    f1 = f1_score(y_true, y_pred_classes, average='weighted')

    results = {
        'train_accuracy': float(history['accuracy'][-1]),
        'val_accuracy': float(history['val_accuracy'][-1]),
        'test_accuracy': float(test_results[1]),
        'final_loss': float(history['loss'][-1]),
        'val_loss': float(history['val_loss'][-1]),
        'test_loss': float(test_results[0]),
        'training_time_seconds': history.get('training_time', 0),
        'completed_epochs': len(history['accuracy']),
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1)
    }

    with open('output_phase1/results.json', 'w') as f:
        json.dump(results, f, indent=4)

    return results


In [ ]:

# تنظیمات GPU
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_GPU_THREAD_COUNT'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# تنظیمات موازی‌سازی
os.environ['TF_NUM_INTRAOP_THREADS'] = str(cpu_count())
os.environ['TF_NUM_INTEROP_THREADS'] = str(cpu_count())
os.environ['OMP_NUM_THREADS'] = str(cpu_count())

# فعال کردن mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# ایجاد پوشه‌های خروجی
os.makedirs('output_phase1/1', exist_ok=True)  # پوشه برای ذخیره نمودارها
os.makedirs('output_phase1/2', exist_ok=True)  # پوشه برای ذخیره تصاویر تست

# لیبل‌های CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

if __name__ == '__main__':
    # بررسی و پیکربندی GPU
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"{len(gpus)} GPU(s) found")
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    # بارگیری داده‌ها
    train_images, train_labels, val_images, val_labels, test_images, test_labels = load_and_preprocess_data()

    # ساخت مدل
    model = build_model()

    # نمایش خلاصه مدل
    model.summary()
    with open('output_phase1/model_summary.txt', 'w') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))

    # آموزش مدل
    start_time = time.time()
    history = train_model(model, train_images, train_labels, val_images, val_labels)
    history['training_time'] = time.time() - start_time

    # ارزیابی روی داده تست
    test_results = model.evaluate(test_images, test_labels, batch_size=512, verbose=0)

    # پیش‌بینی کلاس‌ها برای محاسبه Precision, Recall, F1
    y_pred = model.predict(test_images)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(test_labels, axis=1)

    # رسم نمودارها و ذخیره در پوشه 1
    plot_learning_curves(history)
    plot_confusion_matrix(model, test_images, test_labels)
    plot_roc_curve(model, test_images, test_labels)

    # مصورسازی پیش‌بینی‌ها و ذخیره در پوشه 2
    visualize_predictions(model, test_images, test_labels)

    # ذخیره نتایج
    results = save_results(history, test_results, y_true, y_pred_classes)

    # چاپ تمام معیارها
    print("\nنتایج نهایی:")
    print(f"- دقت آموزش (Train Accuracy): {results['train_accuracy']:.4f}")
    print(f"- دقت اعتبارسنجی (Validation Accuracy): {results['val_accuracy']:.4f}")
    print(f"- دقت تست (Test Accuracy): {results['test_accuracy']:.4f}")
    print(f"- Precision: {results['precision']:.4f}")
    print(f"- Recall: {results['recall']:.4f}")
    print(f"- F1-Score: {results['f1_score']:.4f}")
    print(f"- زمان آموزش: {results['training_time_seconds']:.2f} ثانیه")
    print(f"- تعداد Epochs اجرا شده: {results['completed_epochs']}")

    print("\nآموزش کامل شد. تمام نتایج و نمودارها در پوشه output_phase1 ذخیره شدند.")

1 GPU(s) found
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 4, 4, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 229,066 (894.79 KB)

...

88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9100 - loss: 0.2880 - val_accuracy: 0.6884 - val_loss: 1.0404
Epoch 10/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9311 - loss: 0.2295

88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9311 - loss: 0.2295 - val_accuracy: 0.7068 - val_loss: 1.0142
Epoch 11/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9503 - loss: 0.1760 - val_accuracy: 0.6864 - val_loss: 1.1405
Epoch 12/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9658 - loss: 0.1300 - val_accuracy: 0.6706 - val_loss: 1.2976
Epoch 13/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9778 - loss: 0.0930 - val_accuracy: 0.6774 - val_loss: 1.3009
Epoch 14/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9877 - loss: 0.0647 - val_accuracy: 0.6892 - val_loss: 1.3281
Epoch 15/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9935 - loss: 0.0456 - val_accuracy: 0.6990 - val_loss: 1.2695
Epoch 16/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9967 - loss: 0.0323 - val_accuracy: 0.7046 - val_loss: 1.3122
Epoch 17/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9986 - loss: 0.0204 - val_accuracy: 0.7066 - val_loss:

88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9996 - loss: 0.0123 - val_accuracy: 0.7140 - val_loss: 1.3578
Epoch 19/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 0.0073 - val_accuracy: 0.7116 - val_loss: 1.3784
Epoch 20/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9999 - loss: 0.0049 - val_accuracy: 0.7130 - val_loss: 1.3898
Epoch 21/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 0.0038 - val_accuracy: 0.7136 - val_loss: 1.4207
Epoch 22/30
87/88 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0031

88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 0.7160 - val_loss: 1.4238
Epoch 23/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 0.7148 - val_loss: 1.4477
Epoch 24/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 0.7130 - val_loss: 1.4838
Epoch 25/30
85/88 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0025

88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 0.7162 - val_loss: 1.4792
Epoch 26/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 0.7138 - val_loss: 1.4987
Epoch 27/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 0.7154 - val_loss: 1.5080
Epoch 28/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 0.7158 - val_loss: 1.5276
Epoch 29/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 0.7142 - val_loss: 1.5411
Epoch 30/30
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.7144 - val_loss: 1.5539
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 842ms/step

نتایج نهایی:
- دقت آموزش (Train Accuracy): 1.0000
- دقت اعتبارسنجی (Validation Accu